# Initialization

In [ ]:
# Loading env variables
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# imports
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import ReadTheDocsLoader
from langchain.chains import RetrievalQA

# Running query over ChatGPT

In [ ]:
# Initializing model
llm = OpenAI(temperature=0)
query = "What is Langchain?"

In [ ]:
# Query
print(llm(query))

# Loading documentation

In [ ]:
# Load the documentation
loader = ReadTheDocsLoader("rtdocs", features='html.parser')
docs = loader.load()
len(docs)

In [ ]:
# Size of the one document
len(docs[0].page_content)

In [ ]:
# Splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs_splitted = text_splitter.split_documents(docs)
print(len(docs_splitted))

In [ ]:
# Deduplication
docs_dic = {d.page_content: d for d in docs_splitted}
len(docs_dic)

In [ ]:
# Persistence in Vector DB
import os
persist_directory = 'db'
embeddings = OpenAIEmbeddings()
if os.path.exists(persist_directory + "/index"):
    print("Loading Chroma DB")
    vectorstore = Chroma(embedding_function=embeddings, persist_directory=persist_directory)
else:
    print("Creating Chroma DB")
    vectorstore = Chroma.from_documents(docs_dic.values(), embeddings, persist_directory=persist_directory)
    vectorstore.persist()

# Querying documentation

In [ ]:
# Query docs
docs = vectorstore.as_retriever().get_relevant_documents(query)
docs

In [ ]:
# Generate answer
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff")
chain.run(input_documents=docs, question=query)

In [ ]:
# The same but simpler
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())
qa(query)

In [ ]:
# Next query
query2 = "What is chain?"
qa(query2)